In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [79]:
learning_rate = 0.0001
epochs = 15
batch_size = 16
input_size = 28
max_length = 28
hidden_units = 128
n_class = 10
dropout = 0.8
layers = 1

In [83]:
class LSTM:
    def __init__(self, input_size, layers, hidden_units, max_length, batch_size, learning_rate, dropout):
        with tf.variable_scope("BiLSTM", reuse=tf.AUTO_REUSE):
            self.input_X = tf.placeholder(dtype=tf.float32, shape=[None, None, input_size], name="input_X")
            self.input_y = tf.placeholder(dtype=tf.float32, shape=[None,n_class], name="input_y")
            self.dropout_keep_prob = tf.placeholder(dtype=tf.float32, shape=None, name="dropout_keep_prob")
            self.output = self.bilstm(self.input_X, layers, hidden_units, dropout)
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.input_y, logits=self.output), name="loss")
            self.op_train = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
            self.pred = tf.equal(tf.argmax(self.output,1), tf.argmax(self.input_y,1))
            self.acc = (tf.reduce_mean(tf.cast(self.pred, tf.float32)))*100
    
    def bilstm(self, X, layers, hidden_units, dropout):
        
        fw_cell_stack = []
        for i in range(layers):
            fw_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units)
            fw_cell = tf.nn.rnn_cell.DropoutWrapper(fw_cell, output_keep_prob=dropout)
            fw_cell_stack.append(fw_cell)
        fw_lstm_cell = tf.nn.rnn_cell.MultiRNNCell(fw_cell_stack)
        
        bw_cell_stack = []
        for i in range(layers):
            bw_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units)
            bw_cell = tf.nn.rnn_cell.DropoutWrapper(bw_cell, output_keep_prob=dropout)
            bw_cell_stack.append(bw_cell)
        bw_lstm_cell = tf.nn.rnn_cell.MultiRNNCell(bw_cell_stack)
        
        outputs,_ = tf.nn.bidirectional_dynamic_rnn(fw_lstm_cell, bw_lstm_cell, X, dtype=tf.float32)
        
        fw_output = tf.transpose(outputs[0], perm=[1,0,2])
        fw_output = fw_output[-1]
        bw_output = tf.transpose(outputs[1], perm=[1,0,2])
        bw_output = bw_output[-1]
        
        W = tf.get_variable("W_output",shape=[2*hidden_units,n_class], initializer=tf.random_normal_initializer(), dtype=tf.float32)
        b = tf.get_variable("b_output",shape=[n_class], initializer=tf.zeros_initializer(), dtype=tf.float32)
        output = tf.concat([fw_output,bw_output],axis=1)
        output = tf.add(tf.matmul(output,W), b, name="output")
        return output

In [84]:
with tf.Graph().as_default():
    sess = tf.Session()
    print("Session started")
    model = LSTM(input_size, layers, hidden_units, max_length, batch_size, learning_rate, dropout)
    print("Model was initialized")
    save_path = "./model/LSTM"
    n_batchs = int(mnist.train.num_examples / batch_size)
    saver = tf.train.Saver()
    
    sess.run(tf.global_variables_initializer())
    
    def train_step(batch_X, batch_y, steps):
        feed_dict = {
            model.input_X: batch_X,
            model.input_y: batch_y,
            model.dropout_keep_prob: dropout
        }
        _, loss, acc = sess.run([model.op_train, model.loss, model.acc], feed_dict=feed_dict)
        if steps % 100 == 0:
            print("epochs: {}/{}, iters: {}, loss: {}, acc: {}".format(int(steps/n_batchs)+1,epochs,steps, loss, acc))
    steps = 0
    for i in range(n_batchs*epochs):
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        batch_X = batch_X.reshape((batch_size, max_length, input_size))
        steps += 1
        try:
            train_step(batch_X, batch_y,steps)
        except ValueError:
            print(batch_X)
            print(pre.preprocess(batch_X,max_length).shape)
        except KeyboardInterrupt:
            saver.save(sess,save_path)
            print("Model was saved")
            sys.exit(1)
    saver.save(sess,save_path)
    print("Training finished")
    print("Model was saved")

Session started
Model was initialized
epochs: 1/15, iters: 100, loss: 1.7098276615142822, acc: 50.0
epochs: 1/15, iters: 200, loss: 1.5229506492614746, acc: 43.75
epochs: 1/15, iters: 300, loss: 1.1193931102752686, acc: 75.0
epochs: 1/15, iters: 400, loss: 1.314941167831421, acc: 50.0
epochs: 1/15, iters: 500, loss: 0.8544663190841675, acc: 75.0
epochs: 1/15, iters: 600, loss: 1.2847356796264648, acc: 50.0
epochs: 1/15, iters: 700, loss: 1.0381855964660645, acc: 62.5
epochs: 1/15, iters: 800, loss: 0.9736403226852417, acc: 68.75
epochs: 1/15, iters: 900, loss: 0.5339627265930176, acc: 81.25
epochs: 1/15, iters: 1000, loss: 0.7108867168426514, acc: 81.25
epochs: 1/15, iters: 1100, loss: 0.40899211168289185, acc: 68.75
epochs: 1/15, iters: 1200, loss: 0.5128942728042603, acc: 81.25
epochs: 1/15, iters: 1300, loss: 0.703669011592865, acc: 75.0
epochs: 1/15, iters: 1400, loss: 0.45489293336868286, acc: 75.0
epochs: 1/15, iters: 1500, loss: 1.5824416875839233, acc: 62.5
epochs: 1/15, iters:

SystemExit: 1

/home/jhm9507/miniconda3/envs/jhm/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [85]:
with tf.Graph().as_default():
    sess = tf.Session()
    print("Session started")
    save_path = "./model/LSTM"
    saver = tf.train.import_meta_graph(save_path+".meta")
    saver.restore(sess, save_path)
    graph = tf.get_default_graph()
    print("Model was loaded")
    
    input_X = graph.get_tensor_by_name("BiLSTM/input_X:0")
#     dropout = graph.get_tensor_by_name("BiLSTM/dropout_keep_prob:0")
    output = graph.get_tensor_by_name("BiLSTM/output:0")
    
    test_batch_size = len(mnist.test.images)
    test_xs = mnist.test.images.reshape(test_batch_size, max_length, input_size)
    test_ys = mnist.test.labels
    
    Y = test_ys
    is_correct = tf.equal(tf.argmax(output, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

    print('정확도:', sess.run(accuracy, feed_dict={input_X: test_xs}))

Session started
INFO:tensorflow:Restoring parameters from ./model/LSTM
Model was loaded
정확도: 0.8985
